#Install libraries

In [ ]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install numpy
!pip install datasets
!pip install pytorch_transformers
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip

Import the required libraries

In [ ]:
import torch
from torch.utils.data import (TensorDataset, DataLoader,
                              RandomSampler, SequentialSampler)

from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForSequenceClassification
from pytorch_transformers import AdamW, WarmupLinearSchedule

from distutils.version import LooseVersion as LV

from sklearn.model_selection import train_test_split

import io

import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset

import tensorflow_datasets as tfds

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


sns.set()

if torch.cuda.is_available():
    device = torch.device('cuda')
    devicename = '['+torch.cuda.get_device_name(0)+']'
else:
    device = torch.device('cpu')
    devicename = ""
    
print('Using PyTorch version:', torch.__version__,
      'Device:', device, devicename)
assert(LV(torch.__version__) >= LV("1.0.0"))


ModuleNotFoundError: ignored

Download the IMDb and SST-2 datasets and extract them.

In [ ]:
# Load the IMDB dataset
imdb_dataset = load_dataset("imdb")


# Load the SST-2 dataset
sst2_dataset = load_dataset("glue", "sst2")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Load the IMDb dataset using pandas, and preprocess the text data by removing HTML tags, non-alphanumeric characters, and stop words.

In [ ]:
print(imdb_dataset.column_names)

{'train': ['text', 'label'], 'test': ['text', 'label'], 'unsupervised': ['text', 'label']}


In [ ]:
# Load the IMDb dataset
imdb_df = pd.concat([pd.DataFrame(imdb_dataset['train']),pd.DataFrame(imdb_dataset['test'])])
imdb_df = imdb_df.reset_index(drop=True)


print('\nIMDB data loaded:')
print('data set:', imdb_df.shape)
print(imdb_df['label'].unique())


IMDB data loaded:
data set: (50000, 2)
[0 1]


Load the SST-2 dataset using pandas, and preprocess the text data in the same way as the IMDb dataset

In [ ]:
print(sst2_dataset.column_names)

{'train': ['sentence', 'label', 'idx'], 'validation': ['sentence', 'label', 'idx'], 'test': ['sentence', 'label', 'idx']}


In [ ]:
# Load the SST-2 dataset

sst2_df = pd.concat([pd.DataFrame(sst2_dataset['train'])[['sentence', 'label']],pd.DataFrame(sst2_dataset['validation'])[['sentence', 'label']]])
sst2_df = sst2_df.rename(columns={'sentence': 'text'})
sst2_df = sst2_df.reset_index(drop=True)


print('\nSST2 data loaded:')
print('data set:', sst2_df.shape)
print(sst2_df['label'].unique())



SST2 data loaded:
data set: (68221, 2)
[0 1]


In [ ]:
# Preprocess the text data
sst2_df ['text'] = sst2_df ['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
sst2_df ['text'] = sst2_df ['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
sst2_df ['text'] = sst2_df ['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words

In [ ]:
# Let's view some random reviews:
print(sst2_df.sample(5))

                                                    text  label
21013  sporadic bursts liveliness soso slapstick earp...      1
24055                                   got brawn brains      0
28152                       reflected almost every scene      1
38044                nothing less provocative piece work      1
9434                                       sharply comic      1


Split into train and test set

In [ ]:
# Define your features and target variable
X = sst2_df.drop("label", axis=1)
y = sst2_df["label"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the train and test sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

sst2_train_df = pd.concat([X_train,y_train], axis=1)
sst2_test_df = pd.concat([X_test,y_test], axis=1)

print('\nSST2 data re splitted:')
print('train:', sst2_train_df.shape)
print('test:', sst2_test_df.shape)
print(sst2_train_df['label'].unique())
print(sst2_test_df['label'].unique())


X_train shape: (54576, 1)
X_test shape: (13645, 1)
y_train shape: (54576,)
y_test shape: (13645,)

SST2 data re splitted:
train: (54576, 2)
test: (13645, 2)
[1 0]
[1 0]


In [ ]:
# Let's view some random reviews:
print(sst2_train_df.sample(5))
print(sst2_test_df.sample(5))

                                                    text  label
9365                                                 fun      1
30038  wake saving private ryan black hawk soldiers l...      0
50673           lowkey labor love strikes resonant chord      1
55021                              artsy often pointless      0
19588               make delicate comingofage tale treat      1
                                               text  label
65391  slack execution italicizes absurdity premise      0
2873                          bighearted frequently      1
45196                       bogs surfeit characters      0
67630           chokes depiction uppercrust decorum      0
40568                               sparkling ideas      1


IN-DS: SST2
OOD-DS: IMDB

In [ ]:
#Temporary limit the IN-DS size to 10%
n = 0.05

train_df = sst2_train_df.sample(int(n*sst2_train_df.shape[0]))
test_df = sst2_test_df.sample(int(n*sst2_test_df.shape[0]))

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


n_ood = 0.3
ood_df = imdb_df.sample(int(n_ood*train_df.shape[0]))
ood_df = ood_df.reset_index(drop=True)

In [ ]:
# Preprocess the text data
ood_df ['text'] = ood_df ['text'].str.replace('<.*?>', '', regex=True) # remove HTML tags
ood_df ['text'] = ood_df ['text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True) # remove non-alphanumeric characters
stop_words = set(stopwords.words('english'))
ood_df ['text'] = ood_df ['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])) # remove stop words


# Let's view some random reviews:
print(ood_df.sample(5))

                                                  text  label
778  I say I curious viewing film considered notori...      0
142  A difficult film categorize I never giving 110...      0
740  Loosely based actual events Rivers Edge film m...      1
71   YesIm going 10 heres In last years I watched q...      1
165  I watched film times 90s nearly split sides la...      1


In [ ]:
del X_train, X_test, y_train, y_test

The token `[CLS]` is a special token required by BERT at the beginning of the sentence.

In [ ]:
sentences_train = train_df.text.values
sentences_train = ["[CLS] " + s for s in sentences_train]

sentences_test = test_df.text.values
sentences_test = ["[CLS] " + s for s in sentences_test]

sentences_ood = ood_df.text.values
sentences_ood = ["[CLS] " + s for s in sentences_ood]


labels_train = train_df.label.values
labels_test  = test_df.label.values
labels_ood  = ood_df.label.values

print ("\nThe first training sentence:")
print(sentences_train[0], 'LABEL:', labels_train[0])



The first training sentence:
[CLS] visual merits LABEL: 1


Next we use the BERT tokenizer to convert the sentences into tokens
that match the data BERT was trained on.


In [ ]:
BERTMODEL = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(BERTMODEL,
                                          do_lower_case=True)

tokenized_train = [tokenizer.tokenize(s) for s in sentences_train]
tokenized_test  = [tokenizer.tokenize(s) for s in sentences_test]
tokenized_ood  = [tokenizer.tokenize(s) for s in sentences_ood]

print ("\nThe full tokenized first training sentence:")
print (tokenized_train[0])

print ("\nThe full tokenized first test sentence:")
print (tokenized_test[0])

print ("\nThe full tokenized first OOD sentence:")
print (tokenized_ood[0])


The full tokenized first training sentence:
['[CLS]', 'visual', 'merits']

The full tokenized first test sentence:
['[CLS]', 'funny', 'yet', 'dark', 'seed', '##y', 'clash', 'cultures', 'generations']

The full tokenized first OOD sentence:
['[CLS]', 'a', '2006', 'online', 'poll', 'japan', '##s', 'top', '100', 'favorite', 'animated', 'television', 'series', 'time', 'conducted', 'tv', 'asahi', 'placed', 'series', 'fourth', 'place', 'that', 'tells', 'everything', 'need', 'know', 'so', 'go', 'watch', 'i', 'won', '##t', 'comment', 'story', 'simply', 'i', 'don', '##t', 'want', 'ruin', 'anything', 'i', 'urge', 'keep', 'watching', 'saw', 'first', 'episode', 'the', 'animation', 'really', 'good', 'tv', 'average', 'the', 'best', 'thing', 'series', 'finally', 'something', 'new', 'i', 'mean', 'groundbreaking', 'still', 'offers', 'fresh', 'new', 'idea', 'li', '##ka', '##ble', 'characters', 'you', 'won', '##t', 'regret', 'entering', 'su', '##zumi', '##ya', 'ha', '##ru', '##his', 'world', '##and', 'b


Now we set the maximum sequence lengths for our training and test
sentences as `MAX_LEN_TRAIN` and `MAX_LEN_TEST`. The maximum length
supported by the used BERT model is 512.

The token `[SEP]` is another special token required by BERT at the
end of the sentence.

In [ ]:
MAX_LEN_TRAIN, MAX_LEN_TEST = 128, 512

tokenized_train = [t[:(MAX_LEN_TRAIN-1)]+['SEP'] for t in tokenized_train]
tokenized_test  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_test]
tokenized_ood  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_ood]

print ("\nThe truncated tokenized first training sentence:")
print (tokenized_train[0])


The truncated tokenized first training sentence:
['[CLS]', 'visual', 'merits', 'SEP']



Next we use the BERT tokenizer to convert each token into an integer
index in the BERT vocabulary. We also pad any shorter sequences to
`MAX_LEN_TRAIN` or `MAX_LEN_TEST` indices with trailing zeros.

In [ ]:
ids_train = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_train]
ids_train = np.array([np.pad(i, (0, MAX_LEN_TRAIN-len(i)),
                             mode='constant') for i in ids_train])

ids_test = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_test]
ids_test = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_test])


ids_ood = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_ood]
ids_ood = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_ood])

print ("\nThe indices of the first training sentence:")
print (ids_train[0])


The indices of the first training sentence:
[  101  5107 22617   100     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


BERT also requires *attention masks*, with 1 for each real token in
the sequences and 0 for the padding:

In [ ]:
amasks_train, amasks_test , amasks_ood = [], [] , []

for seq in ids_train:
  seq_mask = [float(i>0) for i in seq]
  amasks_train.append(seq_mask)

for seq in ids_test:
  seq_mask = [float(i>0) for i in seq]
  amasks_test.append(seq_mask)


for seq in ids_ood:
  seq_mask = [float(i>0) for i in seq]
  amasks_ood.append(seq_mask)

We use scikit-learn's train_test_split() to use 10% of our training
data as a validation set, and then convert all data into
torch.tensors.

In [ ]:
(train_inputs, validation_inputs,
 train_labels, validation_labels) = train_test_split(ids_train, labels_train,
                                                     random_state=42,
                                                     test_size=0.1)
(train_masks, validation_masks,
 _, _) = train_test_split(amasks_train, ids_train,
                          random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks  = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks  = torch.tensor(validation_masks)
test_inputs = torch.tensor(ids_test)
test_labels = torch.tensor(labels_test)
test_masks  = torch.tensor(amasks_test)
ood_inputs = torch.tensor(ids_ood)
ood_labels = torch.tensor(labels_ood)
ood_masks  = torch.tensor(amasks_ood)



Next we create PyTorch *DataLoader*s for all data sets.
For fine-tuning BERT on a specific task, the authors recommend a
batch size of 16 or 32.

In [ ]:
BATCH_SIZE = 8

print('\nDatasets:')
print('Train: ', end="")
train_data = TensorDataset(train_inputs, train_masks,
                           train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler,
                              batch_size=BATCH_SIZE)
print(len(train_data), 'reviews')

print('Validation: ', end="")
validation_data = TensorDataset(validation_inputs, validation_masks,
                                validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,
                                   sampler=validation_sampler,
                                   batch_size=BATCH_SIZE)
print(len(validation_data), 'reviews')

print('Test: ', end="")
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler,
                             batch_size=BATCH_SIZE)
print(len(test_data), 'reviews')


print('OOD: ', end="")
ood_data = TensorDataset(ood_inputs, ood_masks, ood_labels)
ood_sampler = SequentialSampler(ood_data)
ood_dataloader = DataLoader(ood_data, sampler=ood_sampler,
                             batch_size=BATCH_SIZE)
print(len(ood_data), 'reviews')


Datasets:
Train: 2455 reviews
Validation: 273 reviews
Test: 682 reviews
OOD: 818 reviews


BERT MODEL INITIALIZATION

We now load a pretrained BERT model with a single linear
classification layer added on top.


In [ ]:
model = BertForSequenceClassification.from_pretrained(BERTMODEL,
                                                      num_labels=2,
                                                      output_hidden_states=True)


model.cuda()
print('\nPretrained BERT model "{}" loaded'.format(BERTMODEL))



Pretrained BERT model "bert-base-uncased" loaded



We set the remaining hyperparameters needed for fine-tuning the
pretrained model: 
 * EPOCHS: the number of training epochs in fine-tuning
   (recommended values between 2 and 4) 
 * WEIGHT_DECAY: weight decay for the Adam optimizer 
 * LR: learning rate for the Adam optimizer 
   (2e-5 to 5e-5 recommended) 
 * WARMUP_STEPS: number of warmup steps to (linearly) reach the
   set learning rate

 We also need to grab the training parameters from the pretrained
 model.

In [ ]:
EPOCHS = 4
WEIGHT_DECAY = 0.01
LR = 2e-5
WARMUP_STEPS =int(0.2*len(train_dataloader))

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)],
     'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS,
                                 t_total=len(train_dataloader)*EPOCHS)

LEARNING

Let's now define functions to train() and evaluate() the model:

In [ ]:
def train(epoch, loss_vector=None, log_interval=200):
    # Set model to training mode
    model.train().to(device)

    # Loop over each batch from the training set
    for step, batch in enumerate(train_dataloader):
        # Copy data to GPU if needed
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

        # Zero gradient buffers
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            # Forward pass
            loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,
                         labels=b_labels)[0]

        if loss_vector is not None:
            loss_vector.append(loss.item())

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()
        scheduler.step()

        # Clear unused variables
        del  b_input_mask, b_labels

        if step % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, step * len(b_input_ids), len(train_dataloader.dataset),
                  100. * step / len(train_dataloader), loss.item()))
            
    # Clear unused variables
    del b_input_ids,batch, loss


In [ ]:
def evaluate(loader):
  model.eval()

  n_correct, n_all = 0, 0

  for batch in loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)
      logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    predictions = np.argmax(logits, axis=1)

    labels = b_labels.to('cpu').numpy()
    n_correct += np.sum(predictions == labels)
    n_all += len(labels)

  print('Accuracy: [{}/{}] {:.4f}'.format(n_correct, n_all,
                                          n_correct/n_all))



    

Now we are ready to train our model using the train()
function. After each epoch, we evaluate the model using the
validation set and evaluate().

In [ ]:
train_lossv = []
for epoch in range(1, EPOCHS + 1):
    print()
    train(epoch, train_lossv)
    print('\nValidation set:')
    evaluate(validation_dataloader)

/usr/local/lib/python3.9/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


Train Epoch: 1 [0/2455 (0%)]	Loss: 0.852963
Train Epoch: 1 [1600/2455 (65%)]	Loss: 0.543897

Validation set:
Accuracy: [209/273] 0.7656

Train Epoch: 2 [0/2455 (0%)]	Loss: 0.141186
Train Epoch: 2 [1600/2455 (65%)]	Loss: 0.387481

Validation set:
Accuracy: [214/273] 0.7839

Train Epoch: 3 [0/2455 (0%)]	Loss: 0.049431
Train Epoch: 3 [1600/2455 (65%)]	Loss: 0.108106

Validation set:
Accuracy: [217/273] 0.7949

Train Epoch: 4 [0/2455 (0%)]	Loss: 0.095080
Train Epoch: 4 [1600/2455 (65%)]	Loss: 0.020699

Validation set:
Accuracy: [220/273] 0.8059


Let's take a look at our training loss over all batches:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_lossv, label='original')
plt.plot(np.convolve(train_lossv, np.ones(101), 'same') / 101,
         label='averaged')
plt.legend(loc='best')
plt.savefig("training-loss.png")
plt.show()


Inference

For a better measure of the quality of the model, let's see the
model accuracy for the test reviews.

In [ ]:
print('\nTest set:')
evaluate(test_dataloader)
# eof


Test set:
Accuracy: [577/682] 0.8460


# OOD detection 

Extract BERT features

In [ ]:
def extract_bert_features(loader):
    model.eval()

    last_layer_list = []
    logits_list = []
    softmax_list = []
    label_list = []
    pred_list = []

    for batch in loader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
            last_layer = outputs[-1]
            logits = outputs[0]
            predictions = np.argmax(logits.detach().cpu().numpy(), axis=1)
            softmax = torch.nn.functional.softmax(logits, dim=-1)

        last_layer_list.append(last_layer)
        logits_list.append(logits.cpu().numpy())
        softmax_list.append(softmax.cpu().numpy())
        label_list.append(b_labels.cpu().numpy())
        pred_list.append(predictions)
    

        

    last_layer = torch.cat(last_layer_list, dim=0)
    logits = np.concatenate(logits_list, axis=0)
    softmax = np.concatenate(softmax_list, axis=0)
    labels = np.concatenate(label_list, axis=0)
    predictions = np.concatenate(pred_list, axis=0)

    return last_layer.to('cpu').numpy(), logits, softmax, labels, predictions
    



Optimized memory version to be tested

In [ ]:
"""

def extract_bert_features(loader):
    model.eval()

    last_layer_list = []
    logits_list = []
    softmax_list = []
    label_list = []
    pred_list = []

    with torch.no_grad():
        for batch in loader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask)
            last_layer = outputs[-1]
            logits = outputs[0].detach().cpu().numpy()
            predictions = np.argmax(logits, axis=1)
            softmax = torch.nn.functional.softmax(outputs[0], dim=-1).cpu().numpy()

            last_layer_list.append(last_layer.cpu())
            logits_list.append(logits)
            softmax_list.append(softmax)
            label_list.append(b_labels.detach().cpu().numpy())
            pred_list.append(predictions)

    last_layer = torch.cat(last_layer_list, dim=0).numpy()
    labels = np.concatenate(label_list, axis=0)
    predictions = np.concatenate(pred_list, axis=0)

    return last_layer.to('cpu').numpy(), logits_list, softmax_list, labels, predictions

    """


In [ ]:
train_features = extract_bert_features(train_dataloader)
test_features = extract_bert_features(test_dataloader)
ood_features = extract_bert_features(ood_dataloader)

OutOfMemoryError: ignored

## OOD using Mahalanobis distance on last layer

Covariance matrix for the latent features of the internal distribution (SST2)

In [ ]:
train_mean = np.mean(train_features, axis=0)
train_cov = np.cov(train_features, rowvar=False)
train_inv_cov = np.linalg.inv(train_cov)


Calculation of the Mahalanobis distance on the in-ds and ood data

In [ ]:
from scipy.spatial.distance import mahalanobis

inds_test_scores = []
ood_test_scores = []

for feature in test_features:
    score = mahalanobis(feature, train_mean, train_inv_cov)
    inds_test_scores.append(score)

for feature in ood_features:
    score = mahalanobis(feature, train_mean, train_inv_cov)
    ood_test_scores.append(score)

We will evaluate the performance of OOD detection using the AUROC, AUPR, FPR and ERR metrics

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve

labels = np.concatenate([np.zeros(len(inds_test_scores)), np.ones(len(ood_test_scores))])
scores = np.concatenate([inds_test_scores, ood_test_scores])

#define the threshold for ood detection
threshold = np.mean(inds_test_scores) +  np.std(inds_test_scores)

In [ ]:
def metrics(scores: np.ndarray, labels: np.ndarray, threshold: float):
    """Computes the number of
        * true positives,
        * false positives,
        * true negatives,
        * false negatives,
        * the accuracy
        * false positive rate
        * taux d'erreur
    for a given threshold

    """
    pos = np.where(scores >= threshold)
    neg = np.where(scores < threshold)
    n_pos = len(pos[0])
    n_neg = len(neg[0])

    tp = np.sum(labels[pos])
    fp = n_pos - tp
    fn = np.sum(labels[neg])
    tn = n_neg - fn

    FPR = fp / (fp + tn)
    Accuracy = (tp+tn)/len(scores)
    ERR = 1 - Accuracy

    return FPR, ERR

FPR, ERR = metrics (scores, labels, threshold)
auroc = roc_auc_score(labels, scores)
aupr = average_precision_score(labels, scores)
print('AUROC:', auroc)
print('AUPR:', aupr)
print('FPR:', FPR)
print('ERR:', ERR)

In [ ]:
fpr, tpr, _ = roc_curve(labels, scores)
precision, recall, _ = precision_recall_curve(labels, scores)

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUROC Curve')

plt.subplot(122)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPR Curve')

plt.tight_layout()
plt.savefig("auroc_aupr_mahalabinos.png")
plt.show()

## OOD detection using Energy-based score
